In [6]:
from datetime import datetime
from canvas_data.api import CanvasDataAPI
import pandas as pd

In [7]:
apiKey = "eb7918aa17275bd8f082e660d35af20c17932963"
apiSecret = "c1015cb4cd91adf9f32f1ba3c6dbb66929259f67"
apiHost = 'portal.inshosteddata.com'
apiContentType = 'application/json'
apiMethod = 'GET'
apiTime = datetime.utcnow().strftime('%a, %d %b %y %H:%M:%S GMT')
schema_latest = '/api/schema/latest'

In [8]:
cd = CanvasDataAPI(api_key=apiKey, api_secret=apiSecret, download_chunk_size=1024*1024)

In [9]:
schema_versions = cd.get_schema_versions()

In [10]:
dumps = cd.get_dumps()

In [11]:
dump_latest = cd.get_schema('latest', key_on_tablenames=True)
norm = pd.json_normalize(dump_latest)

In [12]:
tableN = []
for i in norm.columns:
    x=i.split(".")[0]
    if x  not in tableN:
        tableN.append(x)

In [16]:
tables = pd.read_excel('~/Documents/Canvas Tables.xlsx')
for i,v in enumerate(tables.values):
    tables.loc[i][0]=v[0].strip()

In [21]:
def columns(tableName):
    dump_latest = cd.get_schema('latest', key_on_tablenames=True)
    return pd.json_normalize(dump_latest[tableName]['columns'])

In [19]:
def create_table(schemaName,tableName):
    sql = []
    if 'length' in columns(tableName).columns:
        df = columns(tableName)[['name','type','length']]
        df.length.fillna(0,inplace=True)
        df['length'] = df.length.astype(int)
    else:
        df = columns(tableName)[['name','type']]

    for i,v in enumerate(df.values.tolist()):
        if i == 0:
            sql.append(f"CREATE TABLE IF NOT EXISTS {schemaName}.{tableName} (")
            if len(v) == 3 and v[2] == 0:
                sql.extend(v[:-1])
                sql.append(",")
            else:
                sql.extend(v)
                sql.append(",")
        elif i == len(df.values.tolist())-1:
            if  len(v) == 3 and v[2] == 0:
                sql.extend(v[:-1])
                sql.append(");")
            else:
                sql.extend(v)
                sql.append(");")
        else:
            if len(v) == 3 and v[2] == 0:
                sql.extend(v[:-1])
                sql.append(",")
            else:
                sql.extend(v)
                sql.append(",")
    return print(' '.join(map(str, sql)).upper())

In [22]:
for i,v in enumerate(tables.values):
    create_table('Canvas',v[0].strip())

CREATE TABLE IF NOT EXISTS CANVAS.ASSIGNMENT_FACT ( ASSIGNMENT_ID BIGINT , COURSE_ID BIGINT , COURSE_ACCOUNT_ID BIGINT , ENROLLMENT_TERM_ID BIGINT , POINTS_POSSIBLE DOUBLE PRECISION , PEER_REVIEW_COUNT INT , ASSIGNMENT_GROUP_ID BIGINT , EXTERNAL_TOOL_ID BIGINT );
CREATE TABLE IF NOT EXISTS CANVAS.ASSIGNMENT_GROUP_FACT ( ASSIGNMENT_GROUP_ID BIGINT , COURSE_ID BIGINT , GROUP_WEIGHT DOUBLE PRECISION );
CREATE TABLE IF NOT EXISTS CANVAS.ASSIGNMENT_GROUP_SCORE_FACT ( SCORE_ID BIGINT , CANVAS_ID BIGINT , ACCOUNT_ID BIGINT , COURSE_ID BIGINT , ASSIGNMENT_GROUP_ID BIGINT , ENROLLMENT_ID BIGINT , CURRENT_SCORE DOUBLE PRECISION , FINAL_SCORE DOUBLE PRECISION , MUTED_CURRENT_SCORE DOUBLE PRECISION , MUTED_FINAL_SCORE DOUBLE PRECISION );
CREATE TABLE IF NOT EXISTS CANVAS.COURSE_SCORE_FACT ( SCORE_ID BIGINT , CANVAS_ID BIGINT , ACCOUNT_ID BIGINT , COURSE_ID BIGINT , ENROLLMENT_ID BIGINT , CURRENT_SCORE DOUBLE PRECISION , FINAL_SCORE DOUBLE PRECISION , MUTED_CURRENT_SCORE DOUBLE PRECISION , MUTED_FI

In [23]:
def get_data(tableName,dump_id=None):
    if dump_id==None:
        dump_id='latest'
        local_data_filename = cd.get_data_for_table(table_name=tableName,dump_id=dump_id)
        data = pd.read_csv(local_data_filename, header=None,sep="\t")
        data.columns=columns(tableName)['name'].to_list()
        return data

In [24]:
success_dump=[]
empty_dump=[]
for i,v in enumerate(tables.values):
    try:
        get_data(v[0])
        success_dump.append(v[0])
    except Exception as e:
        empty_dump.append(v[0])

In [25]:
dump_Ids = pd.json_normalize(dumps)['dumpId']

In [ ]:
for i,v  in enumerate(dump_Ids):
    print(v)